In [ ]:
import json
import pandas as pd
from pandas import json_normalize
import pickle
from time import sleep
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import torch
import seaborn as sns
import plotly.express as px
import sklearn
import scipy.stats as stats
from scipy import stats

In [ ]:
file_path = './en_us.json' # Game data containing champion, traits, augemnts items 
with open(file_path, 'r') as json_file:
	data_dragon = json.load(json_file)
data_dragon

In [ ]:
items = pd.DataFrame(data_dragon['items']) # Make item table

game_set = pd.DataFrame(data_dragon['sets']) # filtering only champions and traits: sets8 --> data from season 8

season8_trait = pd.DataFrame(pd.DataFrame(data_dragon['sets']).loc['traits', '8']) 
season8_champ = pd.DataFrame(pd.DataFrame(data_dragon['sets']).loc['champions', '8'])

In [ ]:
# Make dictionary api Name with name that we use in game ex. TFT8_Renekton -> Renekton

item_aug_list = {items.loc[i, 'apiName'] : items.loc[i, 'name'] for i in range(items.shape[0])}
trait_list = {season8_trait.loc[i, 'apiName'] : season8_trait.loc[i, 'name'] for i in range(season8_trait.shape[0])}
champ_list = {season8_champ.loc[i, 'apiName'] : season8_champ.loc[i, 'name'] for i in range(season8_champ.shape[0])}

item_aug_list['None'] = "None"

In [ ]:
game = pd.read_csv('match_record_050422023.csv', index_col=[0])
game


In [ ]:
# Filtering out previous seson (Only take 8.5 seasonl, not 8)
import re
pattern = re.compile(r'Version 13\.6\.499\.7758')
df = game[game['info.game_version'].str.contains(pattern)]

In [ ]:
# Take only participants info. Check the how table looks like
participants = df['info.participants']
participants

In [ ]:
# Necessary for further processing unless occurs error
participants = participants.str.replace("\'", "\"")
participants

In [ ]:
participants = pd.json_normalize(participants.apply(json.loads))
participants

In [ ]:
# Make daytime into 0000/00/00 form
def add_date(game_summary, game_result,index):
    game_id = game_result.iloc[index]["metadata.match_id"]
    date = str(datetime.fromtimestamp(int(game_result.iloc[index]['info.game_datetime'])/1000)).split(' ')[0]
    
    game_summary['Game_id'] = game_id
    game_summary['Datetime'] = date


In [ ]:
def add_augment(game_summary, pariticpant):

    augment = []

    for augments in pariticpant['augments']:

        augments = augments + ['None']*(3-len(augments)) # If no augment: make in 'None'.
        # Make aug dictionary
        aug = {"augment1":item_aug_list[augments[0]],"augment2":item_aug_list[augments[1]],"augment3":item_aug_list[augments[2]]}

        augment.append(aug)
        
    augment_df = pd.DataFrame(dict(augment = augment))

    game_summary[['augments']] = augment_df

In [ ]:
def add_trait(game_summary, participant):
    traits = []

    for i in range(participant.shape[0]):

        check_trait = participant.loc[i, 'traits']
        trait_dict = {}
        traits_list = []
        num_units_list = []
        tier_current_list = []
        for j in check_trait:
            traits_list.append(trait_list[j['name']]) # Trait as Key
            num_units_list.append(j['num_units']) # Number of used units of deck for speciific traits
            tier_current_list.append(j['tier_current']) # Acticated corresponding traits 
        # Make trait dict with  {Trait: {num_units:0, tier:0}}
        for i in range(len(traits_list)):
            trait_dict[traits_list[i]] = {'num_units': num_units_list[i], 'tier_current': tier_current_list[i]}
        traits.append(trait_dict)

    game_summary['traits'] = traits


In [ ]:
def add_unit(game_summary, participant):
    units = []

    for i in range(participant.shape[0]):

        check_unit = participant.loc[i, 'units']
        unit_dict = {}
        units_list = []
        unit_tier_list = []
        item_name_list = []
        for j in check_unit:
            units_list.append(champ_list[j['character_id']]) # Used Champions as key
            item_name_list.append(j['itemNames']) # Used items of correspsonding champion with list form (can contain multiple items)
            unit_tier_list.append(j['tier']) # Tier of corresponding champion
        
        for i in range(len(units_list)):
            unit_dict[units_list[i]] = {'items': item_name_list[i], 'tier_current': unit_tier_list[i]}
        units.append(unit_dict)
        
    game_summary['units'] = units


In [ ]:
# Make first row of participant table
participant_s =  json_normalize(participants.iloc[0,:])
# Take only necessary info
game_summaries = participant_s[['puuid', 'placement', 'time_eliminated', 'last_round', 'total_damage_to_players', 'gold_left']]

game_summary = game_summaries

add_date(game_summaries,game,0)
add_augment(game_summaries, participant_s)
add_trait(game_summaries, participant_s)
add_unit(game_summaries, participant_s)



for num in range(1,participants.shape[0]):
    participant =  json_normalize(participants.iloc[num,:])
    game_summary = participant[['puuid', 'placement', 'time_eliminated', 'last_round', 'total_damage_to_players', 'gold_left']]

    add_date(game_summary,game,num)
    add_augment(game_summary, participant)
    add_trait(game_summary, participant)
    add_unit(game_summary, participant)
    game_summaries = pd.concat([game_summaries,game_summary])

    
# Rearrange columns
colname = ['Player_id', 'Placement', 'Time', 'Last_round', 'Damage_player', 'Gold_left', 'Game_id', 'Datetime', 'augments',  'traits', 'units']
game_summaries.columns = colname
game_summaries = game_summaries[['Datetime', 'Game_id', 'Player_id', 'Placement', 'Time', 'Last_round', 'Damage_player', 'Gold_left', 'augments',  'traits', 'units']]

game_summaries


In [ ]:
# Save table for further usage
game_summaries.to_csv("final_data.csv")

In [ ]:
# Get champion infromation from csv (same info table form eu_us.json(data dragon above))
champ_list = pd.read_csv('champ.csv') 
champ_list   

In [ ]:
# Count champions used in each deck
game_champ_list = {i : [0]*game_summaries.shape[0] for i in list(champ_list["name"])}

for i in range(len(game_summaries['units'])):
    
    units = list(game_summaries.iloc[i]['units'].keys())
    
    for unit in units:
        game_champ_list[unit][i] +=1
    

game_champ_list = pd.DataFrame(game_champ_list)
game_champ_list


In [ ]:
game_champ_list.describe()

In [ ]:
# Check correlation between champions with peason correlation -> Find cloely related champions
check_cor = game_champ_list.corr(method = 'pearson')

check_cor

In [ ]:
correlation_list = {}

correlation_dist = []

first_champ = []

second_champ = []

for i in range(check_cor.shape[0]):
    for j in range(i, check_cor.shape[0]):
        first_champ.append(check_cor.keys()[i])
        second_champ.append(check_cor.keys()[j])
        correlation_dist.append(check_cor.iloc[i][j])

correlation_df = pd.DataFrame(dict(first_champ = first_champ, second_champ = second_champ, cor = correlation_dist))
correlation_df = correlation_df[correlation_df.first_champ != correlation_df.second_champ]

In [ ]:
# Zac is respawned when trait Riftwalker is activated (Vex, Jhin, Pyke) --> Right correlation
correlation_df.sort_values('cor', ascending=False).reset_index(drop=True).head(20)

In [ ]:
px.histogram(data_frame = correlation_df, x='cor')

In [ ]:
def make_contingency_table(dataframe, champion1, champion2):
    
    result = pd.crosstab(dataframe[champion1], dataframe[champion2])
    
    return result

In [ ]:
def make_cramer_v(df, champion1, champion2):
    
    
    dataframe = make_contingency_table(df, champion1, champion2)
    
    x2 = stats.chi2_contingency(dataframe, correction=False)[0]
    N = np.sum(np.sum(dataframe))
    minimum_dimension = min(dataframe.shape)-1

    return(np.sqrt(x2/N)/minimum_dimension)


make_cramer_v(game_champ_list, 'Ashe', 'Renekton')

In [ ]:
correlation_df = correlation_df.reset_index(drop=True)

In [ ]:
check_cramer_v = []

for i in range(correlation_df.shape[0]):
    first_champ = correlation_df.first_champ[i]
    second_champ = correlation_df.second_champ[i]
    
    check_cramer_v.append(make_cramer_v(game_champ_list, first_champ, second_champ))

correlation_df['cramer_v'] = check_cramer_v

correlation_df.sort_values('cramer_v', ascending=False)

In [ ]:
def make_kendall(dataframe, champion1, champion2):
    
    tau, p_value = stats.kendalltau(dataframe[champion1], dataframe[champion2])
    
    return tau

In [ ]:
check_kendall = []

for i in range(correlation_df.shape[0]):
    first_champ = correlation_df.first_champ[i]
    second_champ = correlation_df.second_champ[i]
    
    check_kendall.append(make_kendall(game_champ_list, first_champ, second_champ))
    
correlation_df['kendall'] = check_kendall

correlation_df.sort_values('kendall', ascending=False)

In [ ]:
correlation_df.sort_values('kendall', key= lambda x : abs(x))

In [ ]:
heatmap_df = pd.DataFrame({i : [0]*len(game_champ_list.columns) for i in game_champ_list.columns})

heatmap_df.index = game_champ_list.columns

for i in range(heatmap_df.shape[0]):
    for j in range(heatmap_df.shape[1]):
        first_champ = heatmap_df.index[i]
        second_champ = heatmap_df.columns[j]
        
        kendall = make_kendall(game_champ_list, first_champ, second_champ)
        
        heatmap_df.iloc[i, j] = kendall

px.imshow(heatmap_df, color_continuous_scale = 'RdBu')

In [ ]:
px.imshow(heatmap_df, color_continuous_scale = 'RdBu')

In [ ]:
check_cramer_v = []

for i in range(correlation_df.shape[0]):
    first_champ = correlation_df.first_champ[i]
    second_champ = correlation_df.second_champ[i]
    
    check_cramer_v.append(make_cramer_v(game_champ_list, first_champ, second_champ))

correlation_df['cramer_v'] = check_cramer_v

correlation_df.sort_values('cramer_v', ascending=False)

In [ ]:
px.histogram(data_frame = correlation_df, x='kendall')

In [ ]:
correlation_df.sort_values('cor', ascending=True).reset_index(drop=True).head(20)

In [ ]:
correlation_df.sort_values('cor', ascending=True, key = lambda x : abs(x)).reset_index(drop=True).head(20)


In [ ]:
cor_heatmap = px.imshow(check_cor, color_continuous_scale = 'RdBu')

cor_heatmap

In [ ]:
check = np.array(game_champ_list).astype(float)

check = torch.tensor(check)

svd_u, svd_s, svd_v = torch.svd(check)

In [ ]:
condition_number = svd_s/torch.sum(svd_s)

condition_number = condition_number.numpy()

condition_number_ex = []

for i in range(len(condition_number)):
    condition_number_ex.append(sum(condition_number[:i+1]))
    
scree_ploting=pd.DataFrame(dict(number = range(1, len(condition_number)+1),  condition_number = condition_number_ex))

scree_plot = px.line(data_frame = scree_ploting, x='number', y='condition_number', markers=True)

scree_plot.update_layout(title = 'Scree plot')

# cummulative sum이 0.8을 넘는 지점인 36을 기준으로 선택, 차원을 축소시켰다.

In [ ]:
changed_value = pd.DataFrame(svd_u.numpy()).iloc[:, :36]

In [ ]:
changed_value

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
# 변환된 새로운 값을 이용하여 KMeans를 적용했다. best cluster 수를 찾기 위해 2부터 20까지 within sum of square을 비교했다.

check = []

for i in range(2, 30):
    
    kmeans = KMeans(n_clusters=i, random_state=15).fit(changed_value.iloc[:, 0:36])
    
    check.append(kmeans.inertia_)
    

In [ ]:
find_k = px.line(x= range(2, 30), y=check, markers=True)

In [ ]:
find_k

# 비교 결과 cluster 수가 14 이후 감소율이 작아진다. 따라서 K=14일 때의 결과를 확인했다.

In [ ]:
kmeans = KMeans(n_clusters=8).fit(changed_value)

In [ ]:
from functools import reduce

kmeans = KMeans(n_clusters=8).fit(changed_value)

cluster_check = {'cluster' + str(i+1) : game_champ_list[kmeans.labels_==i] for i in range(8)}

cluster_count = {'cluster' + str(i+1) : pd.DataFrame(cluster_check['cluster'+str(i+1)].apply(sum, 0)) for i in range(8)}

for i in cluster_count.keys():
    cluster_count[i]['champ'] = cluster_count[i].index
    cluster_count[i] = cluster_count[i].reset_index(drop=True)
    cluster_count[i].columns = [i, 'champ']


cluster_result = [cluster_count[i] for i in cluster_count.keys()]

result = reduce(lambda x, y : pd.merge(x, y, on='champ'), cluster_result)

result = result[['champ']+ [f'cluster{i+1}' for i in range(8)]]

result

In [ ]:
result.sort_values('cluster8', ascending=False).reset_index(drop=True).head(20)

In [ ]:
game_summaries

In [ ]:
trait = pd.read_csv('trait.csv')    
trait_list = list(trait["name"])
trait_list

In [ ]:
game_all_list = {i : [0]*game_summaries.shape[0] for i in list(champ_list["name"])}

for i in trait_list:
    game_all_list[i] = [0]*game_summaries.shape[0]


for i in range(len(game_summaries['units'])):

    units = list(game_summaries.iloc[i]['units'].keys())
    
    for unit in units:
        game_all_list[unit][i] +=1

    traits = list(game_summaries.iloc[i]['traits'].keys())
    num_tier = list(game_summaries.iloc[i]['traits'].values())
    tier = []
    for j in num_tier:
        tier.append(list(j.values())[1])
    
    if len(traits)==0:
        pass
    else:
        for k in range(len(traits)):
            game_all_list[traits[k]][i] += tier[k]
    

game_all_list = pd.DataFrame(game_all_list)
game_all_list

In [ ]:
check_all = np.array(game_all_list).astype(float)

check_all = torch.tensor(check_all)

svd_u_all, svd_s_all, svd_v_all = torch.svd(check_all)

condition_number_all = svd_s_all/torch.sum(svd_s_all)

condition_number_all = condition_number_all.numpy()

condition_number_all_ex = []

for i in range(len(condition_number_all)):
    condition_number_all_ex.append(sum(condition_number_all[:i+1]))
    

scree_plot_all = px.line(x=range(1, 98), y=condition_number_all_ex, markers=True)

scree_plot_all.update_xaxes(title = 'singular value number')
scree_plot_all.update_yaxes(title = 'condition number')
scree_plot_all.update_layout(title = 'Scree plot')

scree_plot_all

In [ ]:
changed_value_all = pd.DataFrame(svd_u_all.numpy()).iloc[:, :22]

In [ ]:
changed_value_all

In [ ]:
check_all = []

for i in range(1, 30):
    
    kmeans = KMeans(n_clusters=i).fit(changed_value_all)
    
    check_all.append(kmeans.inertia_)
    
find_k_all = px.line(x= range(1, 30), y=check_all, markers=True)

find_k_all.update_layout(title = 'Within Sum of Square')

find_k_all.update_xaxes(title = 'number of clusters')

find_k_all.update_yaxes(title = 'Within Sum of Square')

In [ ]:
find_k

In [ ]:
game_all_list

In [ ]:
kmeans_all = KMeans(n_clusters=17).fit(changed_value_all)

cluster_check_all = {'cluster' + str(i+1) : game_all_list[kmeans_all.labels_==i] for i in range(17)}

cluster_count_all = {'cluster' + str(i+1) : pd.DataFrame(cluster_check_all['cluster'+str(i+1)].apply(sum, 0)) for i in range(17)}

for i in cluster_count_all.keys():
    cluster_count_all[i]['champ'] = cluster_count_all[i].index
    cluster_count_all[i] = cluster_count_all[i].reset_index(drop=True)
    cluster_count_all[i].columns = [i, 'champ']


cluster_result_all = [cluster_count_all[i] for i in cluster_count_all.keys()]

result_all = reduce(lambda x, y : pd.merge(x, y, on='champ'), cluster_result_all)

result_all = result_all[['champ']+ [f'cluster{i+1}' for i in range(17)]]

result_all

In [ ]:
check_champ = result_all.iloc[:68, :]
check_trait = result_all.iloc[68:, :]

In [ ]:
champ_cluster = check_champ.sort_values('cluster17', ascending=False).reset_index(drop=True)

In [ ]:
trait_cluster = check_trait.sort_values('cluster17', ascending=False).reset_index(drop=True)

In [ ]:
champ_cluster.to_csv("champ_cluster.csv",index=False)
trait_cluster.to_csv("trait_cluster.csv",index=False)

In [ ]:
deck_list= ['Star Guardian Neeko', 'Hacker Draven', 'Quickdraw Lucian', 'Supers Heart', 'RiftWalker Mascot', 'Aegis Threat', 'Hacker Warwick', 'Brawler Warwick', 'Anima Squad', 'Renegade Jhin', 'Ox Force Twisted Fate', 'Supers Underground', 'Duelist', 'LaserCorps Warwick', 'Gadgeteen Gnar', 'Mecha:PRIME Samira', 'InfiniTeam']
deck_result = [deck_list[kmeans_all.labels_[i]] for i in range(len(kmeans_all.labels_))]

game_summaries['Deck'] = deck_result

In [ ]:
game_summaries

In [ ]:
game_summaries.to_csv("final_cluster.csv")

In [ ]:
with open("final_cluster","wb") as f:
    pickle.dump(game_summaries, f)

In [ ]:
champ_list = pd.read_csv('champ.csv') 
index = list(champ_list["name"])

item_set = []

for i in range(len(game_summaries['units'])):

    itemtier = list(game_summaries.iloc[i]['units'].values())
    for j in range(len(itemtier)):
        itemonly = list(itemtier[j].values())[0]
        if len(itemonly) != 0:
            for k in itemonly:
                item_set.append(k)

check_item_usage = {i : [0] for i in set(item_set)}
item_usage = pd.DataFrame(check_item_usage, index=index)
item_usage

In [ ]:
augment_set = []

for i in range(len(game_summaries['augments'])):

    augments = list(game_summaries.iloc[i]['augments'].values())
    for j in augments:
        augment_set.append(j)
    
check_augment_usage = {i : [0] for i in set(augment_set)}
augment_usage = pd.DataFrame(check_augment_usage)

In [ ]:
def deck_select(cluster_name):
    # Make champ item talbe with 0 values
    champ_list = pd.read_csv('champ.csv') 
    index = list(champ_list["name"])

    item_set = []

    for i in range(len(game_summaries['units'])):

        itemtier = list(game_summaries.iloc[i]['units'].values())
        for j in range(len(itemtier)):
            itemonly = list(itemtier[j].values())[0]
            if len(itemonly) != 0:
                for k in itemonly:
                    item_set.append(k)

    check_item_usage = {i : [0] for i in set(item_set)}
    item_usage = pd.DataFrame(check_item_usage, index=index)
   
    #Make augment table with 0 values
    augment_set = []

    for i in range(len(game_summaries['augments'])):

        augments = list(game_summaries.iloc[i]['augments'].values())
        for j in augments:
            augment_set.append(j)
    index_aug = list(set(augment_set))
    check_augment_usage = {"Augment" : [0]}
    #check_augment_usage = {i : [0] for i in set(augment_set)}
    augment_usage = pd.DataFrame(check_augment_usage, index=index_aug)

    df = game_summaries[game_summaries['Deck'] == cluster_name]

    for i in range(len(df['units'])):

        itemtier = list(df.iloc[i]['units'].values())
        champ = list(df.iloc[i]['units'].keys())
        for j in range(len(itemtier)):
            itemonly = list(itemtier[j].values())[0]
            if len(itemonly) != 0:
                for k in itemonly:
                    item_usage.loc[champ[j],k] += 1

    item_usage["Total"] = item_usage.sum(axis=1)
    item_usage_fin = item_usage.sort_values("Total", ascending=False)

    for i in range(len(df['augments'])):

        augments = list(df.iloc[i]['augments'].values())
        for j in augments:
            augment_usage.loc[j,"Augment"] += 1
    
    augment_fin = augment_usage.sort_values("Augment", ascending=False)

    Game_num = len(df["Placement"])

    Total_Placement = 0
    for i in df["Placement"]:
        Total_Placement += i
    Avg_rank = Total_Placement/Game_num #평균 등수

    rank1 = 0
    for i in df["Placement"]:
        if i == 1:
            rank1 += i
    Win_Rate = rank1/Game_num*100 #승률! 1등한거

    rank4 = 0
    for i in df["Placement"]:
        if i <= 4:
            rank4 += 1
    U_rank4 = rank4/Game_num*100 #순방률
    
    return item_usage_fin, augment_fin, Avg_rank, Win_Rate, U_rank4, Game_num


In [ ]:
ItemClus1, AugClus1, Avgrank1, Win1, Urank1, Gamenum1 =deck_select("Star Guardian Neeko")

In [ ]:
ItemClus1.head(6)

In [ ]:
AugClus1.head(10)

In [ ]:
ItemClus2, AugClus2, Avgrank2, Win2, Urank2, Gamenum2 =deck_select("Hacker Draven")
ItemClus3, AugClus3, Avgrank3, Win3, Urank3, Gamenum3 =deck_select("Quickdraw Lucian")
ItemClus4, AugClus4, Avgrank4, Win4, Urank4, Gamenum4 =deck_select("Supers Heart")
ItemClus5, AugClus5, Avgrank5, Win5, Urank5, Gamenum5 =deck_select("RiftWalker Mascot")
ItemClus6, AugClus6, Avgrank6, Win6, Urank6, Gamenum6 =deck_select("Aegis Threat")
ItemClus7, AugClus7, Avgrank7, Win7, Urank7, Gamenum7 =deck_select("Hacker Warwick")
ItemClus8, AugClus8, Avgrank8, Win8, Urank8, Gamenum8 =deck_select("Brawler Warwick")
ItemClus9, AugClus9, Avgrank9, Win9, Urank9, Gamenum9 =deck_select("Anima Squad")
ItemClus10, AugClus10, Avgrank10, Win10, Urank10, Gamenum10 =deck_select("Renegade Jhin")
ItemClus11, AugClus11, Avgrank11, Win11, Urank11, Gamenum11 =deck_select("Ox Force Twisted Fate")
ItemClus12, AugClus12, Avgrank12, Win12, Urank12, Gamenum12 =deck_select("Supers Underground")
ItemClus13, AugClus13, Avgrank13, Win13, Urank13, Gamenum13 =deck_select("Duelist")
ItemClus14, AugClus14, Avgrank14, Win14, Urank14, Gamenum14 =deck_select("LaserCorps Warwick")
ItemClus15, AugClus15, Avgrank15, Win15, Urank15, Gamenum15 =deck_select("Gadgeteen Gnar")
ItemClus16, AugClus16, Avgrank16, Win16, Urank16, Gamenum16 =deck_select("Mecha:PRIME Samira")
ItemClus17, AugClus17, Avgrank17, Win17, Urank17, Gamenum17 =deck_select("InfiniTeam")

In [ ]:
def main_item(ItemClustable,itemlist):
    table = ItemClustable
    table_largest = table.apply(lambda x: x.nlargest(6), axis=1)
    table_largest = table_largest.T
    table_sorted = table_largest.apply(lambda x: x.sort_values().index.tolist())

    Main_champ = list(table_sorted.columns)[:3]
    Main_dict = {}
    main1 = []
    main2 = []
    main3 = []
    for i in range(5):
        main1.append(itemlist[table_sorted.iloc[i][0]])
        main2.append(itemlist[table_sorted.iloc[i][1]])
        main3.append(itemlist[table_sorted.iloc[i][2]])
    Main_dict[Main_champ[0]] = main1[::-1]
    Main_dict[Main_champ[1]] = main2[::-1]
    Main_dict[Main_champ[2]] = main3[::-1]

    return Main_dict

In [ ]:
Meta_deck_dict = {"Deck_Name":[], "MainChamp_and_Item":[], "Main_Augment":[], "Average_Rank":[], "Win_Rate":[], "Under_Rank4":[], "Used_num":[] }
for i in range(1,18):
    item_num = eval("ItemClus" + str(i))
    aug_num = eval("AugClus" + str(i))
    avg_num = eval("Avgrank" + str(i))
    wr_num = eval("Win" + str(i))
    ur_num = eval("Urank" + str(i))
    un_num = eval("Gamenum" + str(i))

    Meta_deck_dict['Deck_Name'].append(deck_list[i-1])
    Meta_deck_dict['MainChamp_and_Item'].append(main_item(item_num,item_aug_list))
    Meta_deck_dict['Main_Augment'].append(list(aug_num.head(10).index))
    Meta_deck_dict['Average_Rank'].append(avg_num)
    Meta_deck_dict['Win_Rate'].append(wr_num)
    Meta_deck_dict['Under_Rank4'].append(ur_num)
    Meta_deck_dict['Used_num'].append(un_num)

Meta_summary = pd.DataFrame.from_dict(Meta_deck_dict)

In [ ]:
Meta_summary = Meta_summary.sort_values("Average_Rank", ascending=True).reset_index(drop=True)

In [ ]:
Meta_summary.to_csv("Meta_deck.csv")